In [10]:
from pyspark import StorageLevel

# Stub code to paste into the shell

sc.setLogLevel("WARN")

# Count web server log requests by user id
userReqs = sc.textFile("/loudacre/weblogs/*2.log")\
  .map(lambda line: line.split()) \
  .map(lambda words: (words[2],1)) \
  .reduceByKey(lambda v1,v2: v1+v2)

# Map account data to (userid,"lastname,firstname") pairs
accounts = sc.textFile("/loudacre/accounts/*")\
  .map(lambda s: s.split(',')) \
  .map(lambda values: (values[0],values[4] + ',' + values[3]))

# Join account names with request counts
accountHits=accounts.join(userReqs)\
  .map(lambda (userid,values): values) 


In [14]:
accountHits.filter(lambda (firstlast, hitcount): hitcount > 5)\
    .count()

5872

In [12]:
accountHits.persist()

PythonRDD[79] at RDD at PythonRDD.scala:43

In [15]:
print accountHits.toDebugString()

(23) PythonRDD[79] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
 |        CachedPartitions: 23; MemorySize: 311.2 KB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B
 |   MapPartitionsRDD[77] at mapPartitions at PythonRDD.scala:374 [Memory Serialized 1x Replicated]
 |   ShuffledRDD[76] at partitionBy at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
 +-(23) PairwiseRDD[75] at join at <ipython-input-10-86bfd874cd9a>:14 [Memory Serialized 1x Replicated]
    |   PythonRDD[74] at join at <ipython-input-10-86bfd874cd9a>:14 [Memory Serialized 1x Replicated]
    |   UnionRDD[73] at union at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
    |   PythonRDD[71] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
    |   /loudacre/accounts/* MapPartitionsRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
    |   /loudacre/accounts/* HadoopRDD[69] at textFile at NativeMethodAccessorImpl.java:

In [16]:
accountHits.unpersist()
accountHits.persist(StorageLevel.DISK_ONLY)

PythonRDD[79] at RDD at PythonRDD.scala:43

In [17]:
accountHits.filter(lambda (firstlast, hitcount): hitcount > 5)\
    .count()

5872